In [54]:
pip install openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.8 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [57]:
import pandas as pd
import requests

# Definir la URL de la API de Banco Mundial y la clave de acceso
world_bank_api_url = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.CD"

# Definir los parámetros de la solicitud
params = {"format": "json", "per_page": 20000, "date": "2010:2020"}

# Realizar la solicitud a la API
response = requests.get(world_bank_api_url, params=params).json()[1]

# Crear una lista de diccionarios con los datos de los países
data = []
for d in response:
    if d["value"] is not None:
        data.append({"country": d["country"]["value"], "year": d["date"], "gdp": d["value"]})

# Crear un DataFrame con los datos de los países
df = pd.DataFrame(data)

# Obtener los datos del tipo de cambio respecto al dólar americano
exchange_rate_url = "https://api.exchangerate-api.com/v4/latest/USD"
exchange_rate_data = requests.get(exchange_rate_url).json()
df["exchange_rate"] = exchange_rate_data["rates"]["MXN"]  # Ejemplo usando el peso mexicano

# Obtener los datos del PIB per cápita
gdp_per_capita_url = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.CD"
gdp_per_capita_params = {"format": "json", "per_page": 20000, "date": "2010:2020"}
gdp_per_capita_response = requests.get(gdp_per_capita_url, params=gdp_per_capita_params).json()[1]

# Crear una lista de diccionarios con los datos del PIB per cápita
gdp_per_capita_data = []
for d in gdp_per_capita_response:
    if d["value"] is not None:
        gdp_per_capita_data.append({"country": d["country"]["value"], "year": d["date"], "gdp_per_capita": d["value"]})

# Crear un DataFrame con los datos del PIB per cápita
gdp_per_capita_df = pd.DataFrame(gdp_per_capita_data)

# Agregar los datos del PIB per cápita al DataFrame principal
df = pd.merge(df, gdp_per_capita_df, on=["country", "year"], how="left")

# Obtener los datos del número de habitantes
population_url = "http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL"
population_params = {"format": "json", "per_page": 20000, "date": "2010:2020"}
population_response = requests.get(population_url, params=population_params).json()[1]

# Crear una lista de diccionarios con los datos del número de habitantes
population_data = []
for d in population_response:
    if d["value"] is not None:
        population_data.append({"country": d["country"]["value"], "year": d["date"], "population": d["value"]})

# Crear un DataFrame con los datos del número de habitantes
population_df = pd.DataFrame(population_data)

# Agregar los datos del número de habitantes al DataFrame principal
df = pd.merge(df, population_df, on=["country", "year"], how="left")

# Mostrar todos los registros del DataFrame

print (df)


                          country  year           gdp  exchange_rate  \
0     Africa Eastern and Southern  2020  9.341791e+11          18.04   
1     Africa Eastern and Southern  2019  1.009052e+12          18.04   
2     Africa Eastern and Southern  2018  1.016697e+12          18.04   
3     Africa Eastern and Southern  2017  1.030482e+12          18.04   
4     Africa Eastern and Southern  2016  8.898593e+11          18.04   
...                           ...   ...           ...            ...   
2825                     Zimbabwe  2014  1.949552e+10          18.04   
2826                     Zimbabwe  2013  1.909102e+10          18.04   
2827                     Zimbabwe  2012  1.711485e+10          18.04   
2828                     Zimbabwe  2011  1.410192e+10          18.04   
2829                     Zimbabwe  2010  1.204166e+10          18.04   

      gdp_per_capita  population  
0        1363.540741   685112705  
1        1512.270553   667242712  
2        1564.734340   6497568

In [58]:
# Guardar el DataFrame como un archivo de Excel
df.to_excel("datos_gdp.xlsx", index=False)

Una vez que descargué los datos en Excel, me di cuanta que tengo información de muchos paises y regiones. para hacer el análisis de este database, procedí a quedarme solamente con variables importantes de latino america y américa del norte para hacer una comparación.

NOTA: la base de datos utilizada no cuenta con argumentos vacíos o erroneos

In [59]:
# Seleccionar solo los países específicos
paises = ["Argentina", "Brazil", "Chile","Mexico","Colombia","Canada","United States"]
df = df[df["country"].isin(paises)]

In [60]:
# Guardar el DataFrame filtrado como un archivo de Excel
df.to_excel("datos_gdp_paises.xlsx", index=False)